In [8]:
!virtualenv bertenv

Using base prefix 'c:\\users\\benar\\anaconda3'
  No LICENSE.txt / LICENSE found in source
New python executable in C:\Users\benar\Desktop\project_data\bertenv\Scripts\python.exe
Installing setuptools, pip, wheel...
done.


In [1]:
# libraries
import os
import csv
import time
import tensorflow as tf
import bert
import pandas as pd
from NYT_parser import NYTArticle
from utilities import *

In [18]:
# get pathways
train_paths = data_setup.get_paths('train')
article = NYTArticle.from_file(os.path.join('data', train_paths[0][0]))

In [20]:
# set destination folder and make folders if needed
dest_folder = 'sample/train/'
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

In [4]:
# attempt a single file output
print("Writing", dest_folder+article.docid)
article.simple_csv_output(dest_folder)

Writing sample/train/739212


In [5]:
# attempt 50 article write minus filtered out
for path in train_paths[:50]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        print("Writing", dest_folder+article.docid)
        article.simple_csv_output(dest_folder)

Writing sample/train/1397928
Writing sample/train/1609775
Writing sample/train/297100
Writing sample/train/448007
Writing sample/train/1463610
Writing sample/train/590096
Writing sample/train/762936
Writing sample/train/1570877
Writing sample/train/1725268
Writing sample/train/1647864
Writing sample/train/1251662
Writing sample/train/551924
Writing sample/train/74723
Writing sample/train/1535917
Writing sample/train/744457
Writing sample/train/97129
Writing sample/train/899133
Writing sample/train/791668
Writing sample/train/779448
Writing sample/train/1287650
Writing sample/train/927546
Writing sample/train/1587613
Writing sample/train/638654
Writing sample/train/870914
Writing sample/train/800527
Writing sample/train/1584905
Writing sample/train/1287373
Writing sample/train/1337862
Writing sample/train/1646102
Writing sample/train/1363692
Writing sample/train/50986
Writing sample/train/1481311
Writing sample/train/1265869
Writing sample/train/199802
Writing sample/train/1000735
Writi

In [14]:
# write all filtered articles in train set
opened_article = 0
written_article = 0
start = time.time()
for path in train_paths[:10000]:
    opened_article += 1
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        written_article += 1
        article.simple_csv_output(dest_folder)
    if opened_article % 10000 == 0:
        print("Opened", opened_article,"articles. Output", written_article,"articles.")
print(f"\n... processed {opened_article} files in {time.time() - start} seconds")

Opened 10000 articles. Output 8341 articles.

... processed 10000 files in 4.4035398960113525 seconds


In [15]:
# build dataframe with articles
sample = 1000

headlines = []
bodies = []

for path in train_paths[:sample]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        article.simple_csv_output(dest_folder)
        headlines.append(article.print_hede[0])
        # join paragraphs for now, may want to do something else later
        bodies.append('\n'.join(article.paragraphs))

train_df = pd.DataFrame(data={'headline':headlines, 'body':bodies})
train_df.head()

,headline,body
0,"A Killer, Yes, but She's a Good Old Girl",Aileen Wuornos was big news 10 years ago. At a...
1,"How High's The Bidding, Mama?",THE blue Rolls Royce sat at the York Street cu...
2,Pacificorp reports earnings for Qtr to Sept 30,LEAD:\n*3*** COMPANY REPORTS **\n*3* Pacificor...
3,Stewart Set to Return,There is bad news for Oakland's competition in...
4,Taxing Online Sales,"To the Editor:\nRe ''So Many Online Sales, So ..."


## Universal Transformer

In [16]:
# Install deps
!pip install -q -U tensor2tensor
!pip install -q tensorflow matplotlib

In [1]:
# Imports we need.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("./t2t/data")
tmp_dir = os.path.expanduser("./t2t/tmp")
train_dir = os.path.expanduser("./t2t/train")
checkpoint_dir = os.path.expanduser("./t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data/"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

W0713 23:38:09.553236 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0713 23:38:11.178983 139783483422528 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0713 23:38:12.607595 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0713 23:38:12.608830 139783483422528 deprecation_wrapper.py:119] From /home/

In [2]:
# problems.available()

In [3]:
# Fetch the problem
sumCNN_problem = problems.problem("summarize_cnn_dailymail32k")

# Copy the vocab file locally so we can encode inputs and decode model outputs
# All vocabs are stored on GCS
# vocab_name = "vocab.translate_ende_wmt32k.32768.subwords"
vocab_name = "vocab.summarize_cnn_dailymail32k.32768.subwords"
vocab_file = os.path.join(gs_data_dir, vocab_name)
!gsutil cp {vocab_file} {data_dir}

# Get the encoders from the problem
encoders = sumCNN_problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers))

Copying gs://tensor2tensor-data/vocab.summarize_cnn_dailymail32k.32768.subwords...
- [1 files][293.7 KiB/293.7 KiB]                                                
Operation completed over 1 objects/293.7 KiB.                                    


W0713 23:38:16.647266 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/text_encoder.py:938: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0713 23:38:16.648737 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/text_encoder.py:940: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [4]:
# Generate and view the data
# This cell is commented out because WMT data generation can take hours

# sumCNN_problem.generate_data(data_dir, tmp_dir)
# example = tfe.Iterator(sumCNN_problem.dataset(Modes.TRAIN, data_dir)).next()
# inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
# targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.

# # Example inputs as int-tensor.
# print("Inputs, encoded:")
# print(inputs)
# print("Inputs, decoded:")
# # Example inputs as a sentence.
# print(decode(inputs))
# # Example targets as int-tensor.
# print("Targets, encoded:")
# print(targets)
# # Example targets as a sentence.
# print("Targets, decoded:")
# print(decode(targets))

In [5]:
# There are many models available in Tensor2Tensor
# NOTE THAT THERE IS A UNIVERSAL TRANSFORMER IN HERE
# registry.list_models()

In [6]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_prepend"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="summarize_cnn_dailymail32k")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
sumCNN_model = registry.model(model_name)(hparams, Modes.TRAIN)

W0713 23:38:16.869588 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/layers/common_layers.py:94: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0713 23:38:17.114422 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/utils/t2t_model.py:243: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



In [7]:
# Copy the pretrained checkpoint locally
# ckpt_name = "transformer_ende_test"
# gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
# !gsutil -q cp -R {gs_ckpt} {checkpoint_dir}
# ckpt_path = tf.train.latest_checkpoint(os.path.join(checkpoint_dir, ckpt_name))
# ckpt_path

In [8]:
# What checkpoints are available
# gs://tensor2tensor-data/
#     gs_data_dir
# !gsutil ls {gs_ckpt_dir}
# !gsutil ls {gs_data_dir}
# !gsutil -q cp -R {'gs://tensor2tensor-data/vocab.cnndailymail.32768'} {checkpoint_dir}
# !gsutil -q cp -R {'gs://tensor2tensor-data/vocab.summarize_cnn_dailymail32k.32768.subwords'} {checkpoint_dir}

In [9]:
# Prepare for the training loop

# In Eager mode, opt.minimize must be passed a loss function wrapped with
# implicit_value_and_gradients
@tfe.implicit_value_and_gradients
def loss_fn(features):
  _, losses = sumCNN_model(features)
  return losses["training"]

# Setup the training data
BATCH_SIZE = 1
sumCNN_problem = problems.problem("summarize_cnn_dailymail32k")
sumCNN_dataset = sumCNN_problem.dataset(Modes.TRAIN, data_dir)
sumCNN_dataset = sumCNN_dataset.repeat(None).batch(BATCH_SIZE)

optimizer = tf.train.AdamOptimizer()

W0713 23:38:17.237405 139783483422528 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/problem.py:680: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
W0713 23:38:17.262322 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/text_problems.py:394: The name tf.VarLenFeature is deprecated. Please use tf.io.VarLenFeature instead.

W0713 23:38:17.263098 139783483422528 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/problem.py:705: The name tf.FixedLenFeature is deprecated. Plea

In [10]:
# Train
NUM_STEPS = 50

for count, example in enumerate(tfe.Iterator(sumCNN_dataset)):
#   print(example["targets"])
#   example["targets"] = tf.reshape(example["targets"], [BATCH_SIZE, 1, 1, 1])  # Make it 4D.
  loss, gv = loss_fn(example)
  optimizer.apply_gradients(gv)

#   if count % 50 == 0:
#     print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
  print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
  if count >= NUM_STEPS:
    break

W0713 23:38:17.462545 139783483422528 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/utils/t2t_model.py:1367: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0713 23:38:17.578155 139783483422528 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/layers/common_attention.py:856: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0713 23:38:17.591538 139783483422528 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/utils/expert_utils.py:621: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0713 23

Step: 0, Loss: 9.538
Step: 1, Loss: 8.869
Step: 2, Loss: 8.981
Step: 3, Loss: 8.170
Step: 4, Loss: 8.577
Step: 5, Loss: 7.643
Step: 6, Loss: 7.989
Step: 7, Loss: 8.202
Step: 8, Loss: 7.758
Step: 9, Loss: 7.898
Step: 10, Loss: 7.992
Step: 11, Loss: 7.677
Step: 12, Loss: 7.712
Step: 13, Loss: 8.245
Step: 14, Loss: 7.242
Step: 15, Loss: 7.520
Step: 16, Loss: 8.657
Step: 17, Loss: 7.090
Step: 18, Loss: 7.319
Step: 19, Loss: 7.459
Step: 20, Loss: 8.075
Step: 21, Loss: 8.205
Step: 22, Loss: 7.676
Step: 23, Loss: 6.971
Step: 24, Loss: 8.007
Step: 25, Loss: 7.403
Step: 26, Loss: 7.468
Step: 27, Loss: 6.904
Step: 28, Loss: 7.520
Step: 29, Loss: 7.623
Step: 30, Loss: 7.588
Step: 31, Loss: 7.643
Step: 32, Loss: 7.620
Step: 33, Loss: 7.318
Step: 34, Loss: 7.433
Step: 35, Loss: 7.667
Step: 36, Loss: 7.077
Step: 37, Loss: 7.985
Step: 38, Loss: 8.073
Step: 39, Loss: 7.463
Step: 40, Loss: 7.257
Step: 41, Loss: 7.105
Step: 42, Loss: 7.541
Step: 43, Loss: 7.835
Step: 44, Loss: 7.481
Step: 45, Loss: 7.94

In [98]:
# inputs = """LEAD: New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'The increase appears to result in part from decisions by state officials to expand social welfare programs such as Medicaid, which provides health care to the poor and is financed jointly by the Federal Government and the states.', 'In the Medicaid program, states set eligibility criteria and define the scope of benefits within guidelines set by Federal law. In the last year many states have expanded Medicaid eligibility or increased benefits to provide more assistance to pregnant women and children from low-income families.', 'In each state, the Federal Government pays at least half the costs of Medicaid and Aid to Families with Dependent Children, the main Federal-state welfare program. Giant Share of U.S. Spending', 'Benefit programs account for nearly half of all Federal spending. In August the budget office estimated that the cost of these programs would rise from $467 billion in the current fiscal year, 1987, to $499 billion in 1988 and eventually to $593 billion in 1991.', 'Aides to Rudolph G. Penner, director of the Congressional Budget Office, said today that the new estimates were $10 billion to $15 billion higher for 1987 and for 1988, and more than $20 billion higher for 1991.', 'These figures suggest that it will be difficult for Congress to meet the deficit targets set in the budget-balancing law signed by President Reagan in December 1985. The targets are $144 billion for 1987 and $108 billion for 1988. Under the law, the budget is supposed to be balanced by 1991.', ""Mr. Reagan plans to submit his 1988 budget to Congress on Monday. His budget director, James C. Miller 3d, has said that the President's budget would show a deficit of no more than $108 billion in 1988. Estimates of the cost of benefit programs have not yet been made public, and the budget does not contain a figure combining all the benefit programs."", 'The latest estimates by the Congressional Budget Office mean that Democrats will be under new pressure to restrain the growth of domestic programs that they might otherwise want to expand. Democrats in the Senate and the House have begun drafting legislation to expand welfare, Medicare, the Federal health insurance program for the elderly and disabled, and Medicaid.', 'Under the major benefit programs, such as Social Security, student loans and unemployment compensation, people are entitled to benefits if they meet certain eligibility requirements set by law. In some of the programs, there is a test of financial need. But in others, people can obtain benefits regardless of their income or asssets.', 'Several of the programs provide assistance to farmers to support the prices of agricultural commodities such as corn, wheat, rice and cotton. Aides to Mr. Penner said the projected costs of these programs had risen substantially above the levels estimated in August: $19.7 billion for 1987 and $21.6 billion for 1988. But they said the new figures would not be disclosed for at least a week. Change in Revenue Estimates', ""Administration officials said President Reagan's budget would show revenue dramatically different from that estimated by Congress when it approved a major tax bill in September."", 'At that time, the Congressional Joint Committee on Taxation compared the new measure with prior law and concluded that it would generate $11 billion more in 1987, but $17 billion less in 1988.', ""But Mr. Reagan's budget, using estimates developed by the Treasury Department, says the new law will produce a bigger gain in 1987, $18 billion, and no loss at all in 1988."", 'David H. Brockway, chief of staff for the joint committee, said the panel had not changed its estimates.', ""The Treasury's figures have political implications because they would make it easier for the President and Congress to reduce the deficit in 1987 and 1988. But in later years the Treasury's estimates would have a different effect, because they show smaller gains in revenue, or more of a loss, than the joint committee's estimates."", 'Most state income-tax laws are linked to Federal tax law in some way. Thirty-five states are expected to gain revenue if they make no change in their own laws. Some of the states plan to use part of that money to expand social welfare benefits.', 'Such changes could require a further increase in Federal spending because the Federal Government shares the cost of several major social welfare programs. The Federal contribution ranges from 50 percent to 78 percent of the cost of benefits under Medicaid and Aid to Families with Dependent Children. Increase for Science Foundation', 'While proposing cuts in many domestic programs, Administration officials said Mr. Reagan would seek an increase of more than 15 percent in the budget of the National Science Foundation, which supports research in science and engineering. The agency has a budget of $1.6 billion this year.', ""The proposed increase is part of Mr. Reagan's overall effort to ''keep America competitive'' by maintaining its technological edge over other countries."", 'But budget officials said Mr. Reagan would ask Congress to slash the budget of the Corporation for Public Broadcasting, which supports public radio and television stations around the country. Congress has already voted to give the corporation $214 million in 1988, $228 million in 1989 and $254 million 1990.', ""Mr. Reagan wants to cut the corporation's budget to $170 million in 1988, $140 million in 1989 and $132 million in 1990, according to confidential budget documents.""]"""

In [12]:
# Restore and translate!
def translate(inputs):
  encoded_inputs = encode(inputs)
#   with tfe.restore_variables_on_create(ckpt_path):
#     model_output = translate_model.infer(encoded_inputs)["outputs"]
  model_output = sumCNN_model.infer(encoded_inputs)["outputs"]
  return decode(model_output)

inputs = "Hello, my name is Ben. And my partner on this project is Mark."
outputs = translate(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

W0713 23:41:38.466648 139783483422528 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/models/transformer.py:1148: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Inputs: Hello, my name is Ben. And my partner on this project is Mark.
Outputs: . . . . . . . . . . . . . . . . . . . . . . . . in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in
